In [1]:
import binary_common as common
import pandas as pd

In [2]:
disorder_labels = [
    "Addictive disorder",
    "Anxiety disorder",
    "Mood disorder",
    "Obsessive compulsive disorder",
    "Schizophrenia",
    "Trauma and stress related disorder",
]

binary_predictions = {}
optimal_thresholds = {}

In [3]:
for label in disorder_labels:
    print(f"\n####### {label} #######\n")
    optimal_thresholds[label] = common.binary_training(label)
    binary_predictions[label] = common.binary_predictions(optimal_thresholds[label], label)
    print('\n\n')


####### Addictive disorder #######

Fitting 5 folds for each of 10 candidates, totalling 50 fits

 Tuned SVM Model for Addictive disorder vs. Healthy control
 Best Parameters: {'C': 0.5, 'kernel': 'linear'}
 Accuracy: 0.6988
 AUC Score: 0.6115
🔹 Optimal Decision Threshold: 0.1859
[1 0 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0
 0 1 1 0 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1
 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1
 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0]
<class 'list'>

Predicted Labels:
 ['Healthy control', 'Addictive disorder', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 

In [5]:
preds = []
for index in range(len(binary_predictions[disorder_labels[0]])):
    max_value = float('-inf')
    max_label = None
    for label in disorder_labels:
        diff = binary_predictions[label][index] - optimal_thresholds[label]
        if diff > 0 and diff > max_value:
            max_value = diff
            max_label = label
    if max_label is None:
        preds.append(common.false_label)
    else:
        preds.append(max_label)

output_df = pd.DataFrame(
        {"y_pred": preds, "y_true": common.read_file("data.csv", common.relevant_columns)["main.disorder"]}
)
common.get_accuracy(output_df)

Accuracy: 0.24


In [6]:
output_df.to_csv('final_output.csv')